# Сложные объединения

## 1. Знакомимся с данными


Таблица city — это справочник городов. Структура справочника представлена ниже.
Название поля 	Тип данных 	Описание
city_id 	    integer 	уникальный идентификатор города, первичный ключ
city_name   	text 	название города
state 	        text 	штат, к которому относится город
population 	    integer 	население города
area 	        numeric 	площадь города

Таблица customer — это справочник клиентов. У компании, с данными которой мы работаем, только корпоративные клиенты, поэтому в таблице нет привычных данных о возрасте и поле. Справочник содержит следующие поля:
Название поля 	Тип данных 	Описание
cust_id 	    integer 	уникальный идентификатор клиента, первичный ключ
cust_name   	text 	    название клиента
annual_revenue 	numeric 	ежегодная выручка
cust_type   	text    	тип пользователя
address 	    text 	    адрес
zip 	        integer 	почтовый индекс
phone       	text    	телефон
city_id        	integer 	идентификатор города, внешний ключ к таблице city

Следующая таблица — driver — справочник водителей. Перечень сведений, содержащихся в таблице, представлен ниже.
Название поля 	Тип данных 	Описание
driver_id 	integer 	уникальный идентификатор водителя, первичный ключ
first_name 	text 	    имя водителя
last_name 	text 	    фамилия водителя
address 	text 	    адрес водителя
zip_code 	integer 	почтовый индекс водителя
phone 	    text 	    телефон водителя
city_id 	integer 	идентификатор города водителя, внешний ключ к таблице city

В таблице truck хранится информация о грузовиках, на которых осуществляются перевозки. Данные о них представлены в следующем виде:
Название поля 	Тип данных 	Описание
truck_id 	integer 	Уникальный идентификатор грузовика, первичный ключ
make 	    text 	    Производитель грузовика
model_year 	integer 	Дата выпуска грузовика

Последняя таблица в датасете, shipment, — таблица с данными непосредственно о доставках. Она описывает взаимодействие всех перечисленных сущностей, а потому содержит наибольшее количество ссылок на другие таблицы.
Название поля 	Тип данных 	Описание
ship_id 	integer 	уникальный идентификатор доставки, первичный ключ
cust_id 	integer 	идентификатор клиента, которому отправлена доставка, внешний ключ к таблице customer
weight 	    numeric 	вес посылки
truck_id 	integer 	идентификатор грузовика, на котором отправлена доставка, внешний ключк таблице truck
driver_id 	integer 	идентификатор водителя, который осуществлял доставку, внешний ключ к таблице driver
city_id 	integer 	идентификатор города в который совершена доставка, внешний ключ к таблице city
ship_date 	date 	    дата доставки

# 2. UNION
## Принцип и условия работы Union
Для этого напишем простой запрос:

SELECT book_name object_name, 'книга' object_descritption /*выбираем столбец с названием book_name, задаём алиас для столбца object_name, задаём во второй колонке объект ‘книга’ с алиасом для столбца object_descritption*/
FROM public.books /*из схемы public и таблицы books*/
UNION ALL /*оператор присоединения*/
SELECT movie_title, 'фильм' /*выбираем столбец movie_title, сами задаём во второй колонке объект ‘фильм’*/
FROM sql.kinopoisk /*из схемы sql и таблицы kinopoisk*/

В запросе мы использовали оператор UNION ALL — он присоединяет любой результат запроса к другому «снизу» при условии, что у них одинаковая структура, а именно:
одинаковый тип данных;
одинаковое количество столбцов;
одинаковый порядок столбцов согласно типу данных.

### Виды UNION

Оператор присоединения существует в двух вариантах:

        UNION выводит только уникальные записи;
        UNION ALL присоединяет все строки последующих таблиц к предыдущим, без ограничений по уникальности.

Важно! UNION оставляет только уникальные значения, а потому требует дополнительных вычислительных мощностей и памяти (в данном случае можно провести аналогию с DISTINCT). Поэтому если вы уверены в отсутствии дубликатов в данных или они вам не важны, предпочтительнее использовать UNION ALL.

### Синтаксис

Запрос строится таким образом:

SELECT
         n columns
FROM 
         table_1

UNION ALL

SELECT 
         n columns
FROM 
         table_2
...
UNION ALL

SELECT 
         n columns
FROM 
         table_n

Результатом выполнения такого запроса будут строки table_1, table_2, ..., table_n, соединённые одни под другими и выведенные в единой выдаче.
Важно! Названия итоговых колонок в выводе будут такие же, как в первом блоке SELECT, даже если они отличаются в других блоках подзапросов.

Пришла пора испытать функцию UNION(ALL) на практике.

Обратимся к нашему датасету о транспортной компании и посмотрим, как сформировать справочник с ID всех таблиц и указанием объекта, к которому он относится.

SELECT
         c.city_id object_name, 'id города' object_type
FROM 
         sql.city c
UNION ALL

SELECT
         d.driver_id other_name, 'id водителя' other_type
FROM 
         sql.driver d
UNION ALL

SELECT
         s.ship_id, 'id доставки'
FROM 
         sql.shipment s
UNION ALL

SELECT
         c.cust_id, 'id клиента'
FROM 
         sql.customer c
UNION ALL

SELECT
         t.truck_id, 'id грузовика'
FROM 
         sql.truck t
ORDER BY 1

Обратите внимание! Несмотря на исходные названия колонок other_name и other_type во втором подзапросе, в выводе мы получим названия, которые дали в первом блоке: object_name и object_type.

Другая особенность — в применении сортировки ORDER BY: она всегда будет относиться к итоговому результату всего запроса с UNION ALL.

В случаях, когда необходимо применить команду ORDER BY или LIMIT не к итоговому результату, а к каждой части запроса, можно обернуть подзапросы в скобки.

Мы уже знаем, что можно легко и непринуждённо применить операторы ORDER BY и LIMIT ко всему результату запроса.

SELECT book_name object_name, 'книга' object_descritption 
FROM public.books
UNION ALL
SELECT movie_title, 'фильм' 
FROM sql.kinopoisk
ORDER BY 1
LIMIT 1

Всё бы хорошо, только в таком случае отсортирован будет весь общий справочник, а в выводе останется одна строка с названием объекта, идущим первым по алфавиту.

А если мы не хотим общую сортировку? Может, нам нужны строки с названием как фильма, так и книги, идущих первыми по алфавиту.

Нет ничего проще — отсортируем каждую часть запроса по отдельности и объединим результаты!

Просто добавим ORDER BY и LIMIT ещё и в первую часть запроса:

SELECT book_name object_name, 'книга' object_descritption 
FROM public.books
ORDER BY 1
LIMIT 1
UNION ALL
SELECT movie_title, 'фильм' 
FROM sql.kinopoisk
ORDER BY 1
LIMIT 1
Выдается ошибка!

Не стоит огорчаться, ведь проблему можно решить одним (ну, почти) движением руки — просто добавив скобки вокруг каждой из частей запроса.

(SELECT book_name object_name, 'книга' object_descritption 
FROM public.books
ORDER BY 1
LIMIT 1)
UNION ALL
(SELECT movie_title, 'фильм' 
FROM sql.kinopoisk
ORDER BY 1
LIMIT 1)

___
Напишите запрос, который создаёт уникальный алфавитный справочник всех городов, штатов, имён водителей и производителей грузовиков.
Результатом запроса должны быть два столбца: название и тип объекта (city, state, driver, truck).
Отсортируйте список по названию объекта, а затем — по типу.

SELECT c.city_name "название", 'city' "тип объекта"
FROM sql.city c
UNION 
SELECT c.state, 'state' 
FROM sql.city c
UNION 
SELECT d.first_name, 'driver' 
FROM  sql.driver d
UNION 
select t.make, 'truck'
 from sql.truck t
 order by 1,2

___
Напишите запрос, который соберёт имена всех упомянутых городов и штатов с таблицы city.
Результатом запроса должен быть один столбец object_name, отсортированный в алфавитном порядке.

SELECT c.city_name object_name
FROM sql.city c
UNION all
SELECT c.state
FROM sql.city c
order by 1

___
Выполнив предыдущий запрос, мы получили города с одинаковыми названиями, но находящиеся в разных штатах, а также большое количество дублирующихся названий штатов.
Перепишите предыдущий запрос так, чтобы остались только уникальные названия городов и штатов.
Результатом запроса должен быть один столбец object_name, отсортированный в алфавитном порядке.

SELECT c.city_name object_name
FROM sql.city c
UNION all
SELECT c.state
FROM sql.city c
order by 1

# 3. UNION и ограничение типов данных

## Почему так важен тип данных?

Важно! Любой тип данных может быть приведён к текстовому формату — эту возможность целесообразно использовать для соединения разнородных сущностей. Главное — помнить, что сортировка текста отличается от сортировки чисел и дат.

SELECT 
        c.city_id::text
FROM
        sql.city c

UNION ALL

SELECT 
        cc.city_name
FROM
        sql.city cc

___
Напишите запрос, который объединит в себе все почтовые индексы водителей и их телефоны в единый столбец-справочник. Также добавьте столбец с именем водителя и столбец с типом контакта (phone или zip в зависимости от типа). Столбцы к выводу: contact, first_name, contact_type.
Отсортируйте список по столбцу с контактными данными в порядке возрастания, а затем — по имени водителя. 

SELECT
    d.zip_code::text contact,
    d.first_name first_name,
    'zip' contact_type
FROM
    sql.driver d
UNION
SELECT
    dd.phone contact,
    dd.first_name first_name,
    'phone' contact_type
FROM
    sql.driver dd
ORDER BY 1,2



## 4. UNION ALL и промежуточные итоги
### Возможности UNION

✍ Помимо соединения разнородных сущностей в единый справочник, UNION ALL часто используется для подведения промежуточных итогов и выведения результатов агрегатных функций.
Кроме агрегатных функций, в запросах с UNION могут использоваться функции группировки и выборки.

Попробуем вывести обобщённые данные о населении по всем городам, с детализацией до конкретного города.

SELECT
         c.city_name,
         c.population
FROM
         sql.city c

UNION ALL

SELECT
         'total',
         SUM(c.population)
FROM
         sql.city c
ORDER BY 2 DESC
__
Напишите запрос, который выводит общее число доставок total_shipments, а также количество доставок в каждый день. Необходимые столбцы: date_period, cnt_shipping.
Не забывайте о единой типизации.
Упорядочьте по убыванию столбца date_period. 

SELECT s.ship_date::text date_period, COUNT(*) cnt_shipment
FROM sql.shipment s 
GROUP BY 1
UNION ALL
SELECT 'total_shipments', COUNT(*)
FROM sql.shipment s
ORDER BY 1 desc

## 5. UNION и дополнительные условия

✍ UNION также может быть использован для разделения существующей выборки по критерию «выполнение определённого условия».

SELECT
         d.first_name,
         d.last_name,
         'телефон заполнен' phone_info
FROM
         sql.driver d
WHERE d.phone IS NOT NULL

UNION

SELECT
         d.first_name,
         d.last_name,
         'телефон не заполнен' phone_info
FROM
         sql.driver d
WHERE d.phone IS NULL

__
Напишите запрос, который выведет все города и штаты, в которых они расположены, а также информацию о том, была ли осуществлена доставка в этот город:

    если в город была осуществлена доставка, то выводим 'доставка осуществлялась';
    если нет — выводим 'доставка не осуществлялась'.

Столбцы к выводу: city_name, state, shipping_status.

Отсортируйте в алфавитном порядке по городу, а затем — по штату.

select
    c.city_name,
    c.state,
    'доставка осуществлялась' shipping_status
from sql.city c 
left join sql.shipment s on c.city_id = s.city_id
WHERE s.ship_id IS NOT NULL
union 
select
    c.city_name,
    c.state,
    'доставка не осуществлялась' shipping_status
from sql.city c 
left join sql.shipment s on c.city_id = s.city_id
WHERE s.ship_id IS  NULL
order by 1,2

__
Напишите запрос, который выводит два столбца: city_name и shippings_fake. Выведите города, куда совершались доставки.

Пусть первый столбец содержит название города, а второй формируется так:

    если в городе было более десяти доставок, вывести количество доставок в этот город как есть;
    иначе — вывести количество доставок, увеличенное на пять.

Отсортируйте по убыванию получившегося «нечестного» количества доставок, а затем — по имени в алфавитном порядке.

SELECT
    c.city_name AS city_name,
    COUNT(s.ship_id) shippings_fake
FROM
    sql.city c
    JOIN sql.shipment s ON c.city_id=s.city_id
GROUP BY
    c.city_name
HAVING
     COUNT(s.ship_id) > 10
UNION
SELECT
    c.city_name AS city_name,
    COUNT(s.ship_id)+5 shippings_fake
FROM
    sql.city c
    JOIN sql.shipment s ON c.city_id=s.city_id
GROUP BY
    c.city_name
HAVING
     COUNT(s.ship_id) <= 10
ORDER BY
    shippings_fake desc,
    city_name asc

## 6. UNION и ручная генерация

✍ UNION можно использовать для создания справочников прямо в коде запроса. К примеру, если мы хотим вручную ввести какие-то значения и произвести с ними некоторые манипуляции или дополнить существующую выдачу своими значениями.

Составим запрос, который позволит вывести первые три буквы алфавита и их порядковые номера.

SELECT 
         'a' letter,'1' ordinal_position /*сами задаём значение первого столбца ‘a’ и алиас для него letter, значение второго столбца ‘1’ и алиас для него ordinal_position*/
         
UNION /*оператор присоединения*/

SELECT 
         'b','2' /*сами задаём значение первого столбца ‘b’, значение второго столбца ‘2’ */
         
UNION /*оператор присоединения*/

SELECT
         'c','3' /*сами задаём значение первого столбца ‘с’, значение второго столбца ‘3’*/ 

__
Напишите запрос, который выберет наибольшее из значений:

    1000000;
    541;
    -500;
    100.

SELECT  1000000
UNION ALL
SELECT  541
UNION ALL
SELECT  -500
UNION ALL
SELECT  100
ORDER BY 1 DESC 
LIMIT 1

__
Мы помним, что сортировка для числовых и строковых типов данных отличается.

Построив запрос по аналогии с примером, приведите значения к текстовому типу данных, сравните и выберите наибольшее из них:
SELECT  '1000000'
UNION ALL
SELECT  '541'
UNION ALL
SELECT  '-500'
UNION ALL
SELECT  '100'
order by 1 desc
limit 1

__
Построив запрос по аналогии с примером, найдите самое большое значение из перечисленных операторов:

    + ;
    - ;
    = ;
    / .

SELECT  '+'
UNION ALL
SELECT  '-'
UNION ALL
SELECT  '='
UNION ALL
SELECT  '/'
order by 1 desc
limit 1

## 7. EXCEPT

### Исключаем повторяющиеся данные



Предположим, нам нужно узнать, в какие города осуществлялась доставка, за исключением тех, в которых проживают водители.

SELECT
         c.city_name
FROM
         sql.shipment s
JOIN sql.city c ON s.city_id = c.city_id

EXCEPT

SELECT
         cc.city_name
FROM
         sql.driver d 
JOIN sql.city cc ON d.city_id=cc.city_id
ORDER BY 1
Все водители проживают в городе Memphis, и мы видим, что он не выводится в результате запроса.

Синтаксические правила для оператора EXCEPT такие же, как и для UNION:

        одинаковый тип данных;
        одинаковое количество столбцов;
        одинаковый порядок столбцов согласно типу данных.

Синтаксис выглядит следующим образом:
SELECT 
         n columns
FROM 
         table_1

EXCEPT

SELECT 
         n columns
FROM 
         table_2
___
Выведите список zip-кодов, которые есть в таблице sql.driver, но отсутствуют в таблице sql.customer. Отсортируйте по возрастанию, столбец к выводу — zip.
В поле ниже введите запрос, с помощью которого вы решили эту задачу. 

SELECT
         d.zip_code
FROM
         sql.driver d
JOIN sql.customer c ON d.city_id = c.city_id

EXCEPT

SELECT
         cc.zip
FROM
         sql.driver d 
JOIN sql.customer cc ON d.city_id=cc.city_id
ORDER BY 1


## 8. INTERSECT

### Выбираем общие данные

А что если нам надо вывести общие записи — те, что существуют в нескольких таблицах?

Предположим, нам надо вывести совпадающие по названию города и штаты.\

SELECT 
         c.city_name object_name
FROM 
         sql.city c

INTERSECT

SELECT 
         cc.state
FROM 
         sql.city cc
ORDER BY 1

__
Напишите запрос, который выведет список id городов, в которых есть и клиенты, и доставки, и водители. 
select
    c.city_id object_name
from sql.city c
intersect
select cc.city_id 
from sql.customer cc
intersect
select
    s.city_id
from sql.shipment s
intersect
select
    d.city_id
from sql.driver d

__
Выведите zip-код, который есть как в таблице с клиентами, так и в таблице с водителями. 

select
    zip object_name
from sql.customer
intersect
select
    zip_code
from sql.driver



## Итоги

Выведите города с максимальным и минимальным весом единичной доставки.
Столбцы к выводу — city_name, weight.

(SELECT
	   c.city_name,
	   s.weight
	FROM sql.shipment s
	   JOIN sql.city c ON s.city_id=c.city_id
	ORDER BY 2 desc
	LIMIT 1)
UNION ALL
(SELECT
	   c.city_name,
	   s.weight
	FROM sql.shipment s
	   JOIN sql.city c ON s.city_id=c.city_id
	ORDER BY 2
	LIMIT 1)	

__
Выведите идентификационные номера клиентов (cust_id), которые совпадают с идентификационными номерами доставок (ship_id).
Столбец к выводу — mutual_id.
Отсортируйте по возрастанию.

select
    c.cust_id as mutual_id
from sql.customer c
intersect
select
    s.ship_id
from sql.shipment s
order by 1

__
Создайте справочник, содержащий уникальные имена клиентов, которые являются производителями (cust_type='manufacturer'), и производителей грузовиков, а также описание объекта — 'КЛИЕНТ' или 'ГРУЗОВИК'.
Столбцы к выводу — object_name, object_description.
Отсортируйте по названию в алфавитном порядке. 

select
    cust_name object_name, 'КЛИЕНТ' object_description
from sql.customer 
where cust_type='manufacturer'
union
select
    make, 'ГРУЗОВИК'
from sql.truck 
order by 1